In [2]:
import pandas as pd
import ast

# Optimización de dataset

Auquí nos centraremos en optimizar los datasets para, obtener nuevos con fin de mejorar el rendimiento de las API

In [4]:
# cargamos los conjuntos de datos que preparamos en el ETL
steam_games = pd.read_csv('../datasets/steam_games.csv')
user_reviews = pd.read_parquet('../datasets/user_reviews.parquet')
users_items = pd.read_parquet('../datasets/users_items.parquet')

In [5]:
steam_games.columns

Index(['genres', 'app_name', 'price', 'item_id', 'developer', 'año'], dtype='object')

In [6]:
users_items.columns

Index(['user_id', 'steam_id', 'item_id', 'horas'], dtype='object')

## PlayTimeGenre

In [9]:
# Convertir la columna 'item_id' a tipo int64 en ambos DataFrames
users_items['item_id'] = users_items['item_id'].astype('int64')
steam_games['item_id'] = steam_games['item_id'].astype('int64')

merge = pd.merge(users_items, steam_games[['item_id', 'año', 'genres']], on='item_id', how='inner')

# Ahora haremos un groupby por genero y año de lanzamiento
merge = merge.groupby(['año', 'genres'])['horas'].sum().reset_index()

#Convertiremos la columna genres a lista
merge['genres'] = merge['genres'].apply(ast.literal_eval)

#Pasamos las listas de genres a columnas
merge = merge.explode('genres').reset_index(drop=True)

merge

,año,genres,horas
0,1983,Action,57.883333
1,1983,Adventure,57.883333
2,1983,Casual,57.883333
3,1983,FMV,57.883333
4,1983,Quick-Time Events,57.883333
...,...,...,...
69044,2018,Early Access,1.700000
69045,2018,Platformer,1.700000
69046,2018,Metroidvania,1.700000
69047,2018,RPG,63.350000


In [10]:
# Encontrar el género con más horas jugadas por año 
genre = merge.loc[merge.groupby(['año', 'genres'])['horas'].idxmax()]

# Ordenar el DataFrame resultante
genre = genre.sort_values(by=['genres', 'horas', 'año'], ascending=[True, False, True])

# Guardar el resultado en un archivo CSV
genre.to_csv('../datasets/año_genres.csv', index=False)